In [129]:
from datetime import datetime, timezone
import configparser
import json
import requests
import ccxt
import time
from pprint import pprint 
import pandas as pd
from urllib.parse import urljoin 



binance_symbol = 'BTC/USDT'
coinbase_symbol = 'BTC/USD'

coinbase_datetime_format = '%Y-%m-%dT%H:%M:%S'
default_datetime_format = '%Y-%m-%d %H:%M:%S'

binance_data_columns = ['time', 'open', 'high', 'low', 'close', 'volume']
coinbase_data_columns = ['time', 'low', 'high', 'open', 'close', 'volume']


def table(values):
    first = values[0]
    keys = list(first.keys()) if isinstance(first, dict) else range(0, len(first))
    widths = [max([len(str(v[k])) for v in values]) for k in keys]
    string = ' | '.join(['{:<' + str(w) + '}' for w in widths])
    return "\n".join([string.format(*[str(v[k]) for k in keys]) for v in values])

def listdata_to_df(data, cols):
    df = pd.DataFrame(data, columns = cols)
    # df['time'] = [binance_ts_to_datetime(time) for time in df['time']]
    # df.set_index('Time', inplace=True)
    return(df)

def coinbase_data_to_df(data):
    return(listdata_to_df(data, cols=coinbase_data_columns))

def binance_data_to_df(data):
    return(listdata_to_df(data, cols=binance_data_columns))

def ts_to_dt(ts, tzinfo='korea'):     
    # ts_binance_size = 13

    # KST = datetime.timezone(datetime.timedelta(hours=9))
    if tzinfo == 'binance':
        return(datetime.utcfromtimestamp(float(ts)/1000)) 
    elif tzinfo == 'coinbase':
        return(datetime.utcfromtimestamp(float(ts)))
    elif tzinfo == 'korea':
        return(datetime.fromtimestamp(float(ts)))

def dt_to_str(dt, format):
    return(dt.strftime(format))

def binance_ts_to_dt(time):
    return(datetime.utcfromtimestamp(float(time)/1000))

# def binance_data_to_df(data):
#     df = pd.DataFrame(data, columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
#     df['Time'] = [binance_ts_to_dt(time) for time in df['Time']]
#     df.set_index('Time', inplace=True)
#     return(df)

def binance_ts_to_coinbase_datestr(ts, format=coinbase_datetime_format):
    return(dt_to_str(binance_ts_to_dt(ts), format))



# def date_from_timestamp(timestamp):
#     return(timestamp/1000)


config = configparser.ConfigParser()
config.read('../config.ini')

BINANCE_API_KEY = config['BINANCE']['API_KEY']
BINANCE_SECRET_KEY = config['BINANCE']['SECRET_KEY']
BINANCE_API_URL = config['BINANCE']['API_URL']

COINBASE_API_KEY = config['COINBASE']['API_KEY']
COINBASE_SECRET_KEY = config['COINBASE']['SECRET_KEY']
COINBASE_API_URL = config['COINBASE']['API_URL']

print('CCXT Version: ', ccxt.__version__)
exchange_id = 'binance'
binance = ccxt.binance({
    'options': {
        'adjustForTimeDifference': True,
        'defaultType': 'future'
    },
    'rateLimit': 10000,
    'enableRateLimit': True,
    # 'verbose': True,
    'apiKey': BINANCE_API_KEY,
    'secret': BINANCE_SECRET_KEY,
})

coinbase = ccxt.coinbase({
    'options': {
        'adjustForTimeDifference': True,
        'defaultType': 'future'
    },
    'rateLimit': 10000,
    'enableRateLimit': True,
    # 'verbose': True,
    'apiKey': COINBASE_API_KEY,
    'secret': COINBASE_SECRET_KEY,
})




CCXT Version:  1.45.42


In [ ]:
# pprint(binance.api)
# pprint(binance.has)
# pprint(binance.timeframes)

# pprint(coinbase.api)
# pprint(coinbase.has)
# pprint(coinbase.timeframes)

# if binance.has['fetchOHLCV']:
#     time.sleep (binance.rateLimit / 1000) # time.sleep wants seconds
#     one_day = binance.fetch_ohlcv(symbol, '1d')


# binance returns 500 data per query 
# len(one_day)


In [143]:
def ohlcv_data(symbol, exchange, timeframe, since=None):
    msec = 1000
    minute = 60 * msec
    hour = 60 *minute
    day = 24 * hour

    if timeframe == '1m':
        tick = minute
    elif timeframe == '1h':
        tick = hour
    elif timeframe == '1d':
        tick = day 

    data = []
    now_timestamp = exchange.milliseconds()
    since_timestamp = exchange.parse8601(since)

    print('Difference: ', (now_timestamp - since_timestamp)/minute, '(',now_timestamp, '-', since_timestamp, ')')

    if (since_timestamp == None or since_timestamp > now_timestamp):
        print('Fetching candles starting from: ', since)
        data = exchange.fetch_ohlcv(symbol, timeframe)
        print('Fetching finished')  
        return(data)


    while since_timestamp < now_timestamp: 
        print('Fetching candles starting from: ', binance_ts_to_dt(since_timestamp))
        candles = exchange.fetch_ohlcv(symbol, timeframe, since=since_timestamp)
        since_timestamp = candles[-1][0] + tick 
        data += candles

    print('Fetching finished')
    return(data)

# def binance_timestamp_to_datetime(time):
#     return(datetime.utcfromtimestamp(float(time)/1000))


# def binance_data_to_dataframe(data):
#     df = pd.DataFrame(data, columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
#     df['Time'] = [binance_timestamp_to_datetime(time) for time in df['Time']]
#     df.set_index('Time', inplace=True)
#     return(df)


# def ts_to_dt(ts, tzinfo='korea'):     
#     # ts_binance_size = 13

#     # KST = datetime.timezone(datetime.timedelta(hours=9))
#     if tzinfo == 'binance':
#         return(datetime.utcfromtimestamp(float(ts)/1000)) 
#     elif tzinfo == 'coinbase':
#         return(datetime.utcfromtimestamp(float(ts)))
#     elif tzinfo == 'korea':
#         return(datetime.fromtimestamp(float(ts)))

# def dt_to_str(dt, format):
#     return(dt.strftime(format))

# def binance_ts_to_coinbase_datestr(ts, format=coinbase_datetime_format):
#     return(dt_to_str(binance_timestamp_to_datetime(ts), format))


def coinbase_data_fetch(start, stop, granularity=60):

    PATH = 'products/BTC-USD/candles'
    # COINBASE_URL = urljoin(COINBASE_API_URL, PATH)

    data = []
    #PATH = 'btc/exchange-flows/reserve'
    # URL = urljoin(API_URL, PATH)
    params = {
        'start': start,
        'stop': stop,
        'granularity': granularity
    }

    response = requests.get(urljoin(COINBASE_API_URL, PATH), params=params).json()
    # print(len(str(response[0][0])))
    return(response)

    

In [126]:
COINBASE_API_URL

"'https://api.pro.coinbase.com/'"

In [119]:
dt_now = datetime.now()
print('dt_now:', dt_now)


ts_now = time.time()
ts_binance = binance.milliseconds()
print('ts_now:', ts_now)
print('ts_binance:', ts_binance)

dt_binance = ts_to_dt(ts_binance, 'binance')
dt_coinbase = ts_to_dt(ts_binance/1000, 'coinbase')
dt_korea = ts_to_dt(ts_now)
print('dt_korea:', dt_korea)
print('dt_binance:', dt_binance)
print('dt_coinbase:', dt_coinbase)



dt_now: 2021-04-23 15:43:56.359554
ts_now: 1619160236.3602376
ts_binance: 1619160236360
dt_korea: 2021-04-23 15:43:56.360238
dt_binance: 2021-04-23 06:43:56.360000
dt_coinbase: 2021-04-23 06:43:56.360000


In [120]:
test_data = ohlcv_data(symbol, binance, '1m', since='2021-04-23 00:00:00')

Difference:  403.9714 ( 1619160238284 - 1619136000000 )
Fetching candles starting from:  2021-04-23 00:00:00
Fetching finished


In [131]:
print(len((test_data)))
print('difference:', (test_data[-1][0] - test_data[0][0]), '(', test_data[-1][0], '-',  test_data[0][0],  (test_data[-1][0] - test_data[0][0])/60000 +1)

405
difference: 24240000 ( 1619160240000 - 1619136000000 405.0


In [132]:
# test_data

In [133]:
start = binance_ts_to_coinbase_datestr(test_data[0][0])
stop = binance_ts_to_coinbase_datestr(test_data[-1][0])

print(start, end)

2021-04-23T00:00:00 2021-04-23T03:00:00


In [134]:
len(test_data)

405

In [146]:

COINBASE_URL = 'https://api.pro.coinbase.com/products/BTC-USD/candles'

#PATH = 'btc/exchange-flows/reserve'
# URL = urljoin(API_URL, PATH)
params = {
    'start': start,
    'stop': stop,
    'granularity': 60
}

response = requests.get(COINBASE_URL, params=params).json()
print(len(response))

300


In [144]:

coinbase_data = coinbase_data_fetch(start, stop)
print('length: ', len(coinbase_data))
print(coinbase_data)


# since='2021-04-18 00:00:00'

# # stop = datetime.strftime(since, format)
# # stop


# # COINBASE_URL = 'https://api.pro.coinbase.com/products/BTC-USD/candles'

# #PATH = 'btc/exchange-flows/reserve'
# # URL = urljoin(API_URL, PATH)
# params = {
#     'start': start,
#     'stop': stop,
#     'granularity': 60
# }

# response = requests.get(COINBASE_API_URL, params=params).json()
# # print(response)
# # print(response['status'])
# # print(response['result'])
# # print(len(response))
# print(len(str(response[0][0])))

InvalidSchema: No connection adapters were found for "'https:/api.pro.coinbase.com/products/BTC-USD/candles"

In [94]:
type(response)

list

In [95]:
df = binance_data_to_df(test_data)

print(len(df))

389


In [47]:
df
# print(df.iloc[-1, :])
# print(df.iloc[-1, 0])
# print(df.iloc[-1, 1])
# print(df.iloc[-1, 2])
# print(df.iloc[-1, 3])
# print(df.iloc[-1, 4])

# print(df[-1:].index)

,Open,High,Low,Close,Volume
Time,,,,,
2021-04-23 00:00:00,51714.62,52152.00,51288.65,51568.91,17629.095
2021-04-23 01:00:00,51568.91,51771.60,49530.04,51085.04,45551.807
2021-04-23 02:00:00,51085.04,51476.39,50352.25,50440.53,9179.742


In [51]:
ts_to_dt(test_data[-1][0]/1000, 'coinbase')

datetime.datetime(2021, 4, 23, 2, 0)

In [ ]:
coinbase_symbol = 'BTC/USD'
coinbase.load_markets()

In [52]:
dt_start = binance_timestamp_to_datetime(test_data[0][0])
dt_end = binance_timestamp_to_datetime(test_data[-1][0])

In [53]:
print(dt_start)
print(dt_end)

2021-04-23 00:00:00
2021-04-23 02:00:00


In [43]:
# def coinbase_ohlcv_data(symbol, timeframe, start, stop):
type(df[-1:].index)


pandas.core.indexes.datetimes.DatetimeIndex

In [55]:

# 2018-07-10T12:00:00 
coinbase_time_format = '%Y-%m-%dT%H:%M:%S'
default_time_format = '%Y-%m-%d %H:%M:%S'


print(dt_to_str(dt_start, coinbase_time_format))

2021-04-23T00:00:00


In [ ]:





since='2021-04-18 00:00:00'

# stop = datetime.strftime(since, format)
# stop


COINBASE_URL = 'https://api.pro.coinbase.com/products/BTC-USD/candles'

#PATH = 'btc/exchange-flows/reserve'
# URL = urljoin(API_URL, PATH)
params = {
    'start': '2018-07-10T12:00:00',
    'stop': '2018-07-15T12:00:00',
    'granularity': 60
}

response = requests.get(COINBASE_URL, params=params).json()
print(response)
# print(response['status'])
# print(response['result'])
print(len(response))

In [40]:
COINBASE_URL = 'https://api.pro.coinbase.com/time'

#PATH = 'btc/exchange-flows/reserve'
# URL = urljoin(API_URL, PATH)
# params = {
#     'start': '2018-07-10T12:00:00',
#     'stop': '2018-07-15T12:00:00',
#     'granularity': 60
# }

response = requests.get(COINBASE_URL).json()
print(response)



{'iso': '2021-04-23T02:50:50.032Z', 'epoch': 1619146250.032}
